In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import leafmap.foliumap as leafmap
import folium
import altair as alt
from scipy.stats import linregress
import matplotlib.colors as mcolors
import scipy.stats as stats


In [2]:
alt.data_transformers.enable('default', max_rows = None)

DataTransformerRegistry.enable('default')

In [3]:
def create_vegalite_chart(df):

    x = np.arange(df.shape[0])
    y = df['val'].values

    chart = alt.Chart(df).mark_line().encode(
        alt.X('data'),
        alt.Y('val', scale=alt.Scale(zero=False))
    )

    if df.empty:
        return alt.Chart().mark_line()
    

    slope, intercept, r_value, p_value, std_err = linregress(x, y)

    regression_line = intercept + (x * slope)
    regression_df = pd.DataFrame({'data': x, 'regression': regression_line})

    regression_chart = alt.Chart(regression_df).mark_line(color='red').encode(
        alt.X('data'),
        alt.Y('regression')
    )

    return chart + regression_chart

In [20]:
def get_color(df):
    if df.empty:
        return 'black', 1

    x = np.arange(df.shape[0])
    y = df['val'].values
    slope, _, _, _, _ = linregress(x, y)

    # column = df['val']
    # min_val = column.min()
    # max_val = column.max()
    # norm_val = (column - min_val)/(max_val-min_val)
    # transp = norm_val * 2 - 1

    if slope > 0:
        return 'green', slope
    elif slope < 0:
        return 'red', slope
    else:
        return 'blue', 1

In [5]:
anagraf_folder_path = "C:\\Users\\Andrew\\Desktop\\gw_study\\girotto_lab\\processed_data\\All_Anagrafica\\"
anagraf_df = pd.DataFrame()

for file in os.listdir(anagraf_folder_path):
    df = pd.read_excel(anagraf_folder_path + "\\" + file)
    anagraf_df = anagraf_df.append(df)

df_subset = anagraf_df[['Longitude', 'Latitude', 'Codice']].dropna(subset=['Latitude', 'Longitude'])

In [6]:
complete_df = pd.DataFrame()

for file in os.listdir('C:\\Users\\Andrew\\Desktop\\gw_study\\girotto_lab\\processed_data\\all_combined'):
    df = pd.read_excel('C:\\Users\\Andrew\\Desktop\\gw_study\\girotto_lab\\processed_data\\all_combined\\' + file)
    complete_df = complete_df.append(df)

In [7]:
complete_df = complete_df[complete_df['val'] != '---']
complete_df['val'] = complete_df['val'].astype(float)
complete_df = complete_df[complete_df['val'].notna()]

In [10]:
grouper = complete_df.groupby(['codice', pd.Grouper(key='data', freq='MS')])['val'].mean().reset_index()


,codice,data,val
0,1,1999-08-01,2.93
1,1,1999-09-01,2.78
2,1,2000-01-01,2.89
3,1,2000-02-01,2.54
4,1,2000-10-01,2.79
...,...,...,...
90925,Ver6,2014-01-01,8.20
90926,Ver6,2014-02-01,7.99
90927,Ver6,2014-03-01,7.25
90928,Ver6,2014-05-01,6.85


In [11]:
complete_df = grouper

In [21]:
m = folium.Map(location=[df_subset['Latitude'].mean(), df_subset['Longitude'].mean()], zoom_start=6)

for _, row in df_subset.iterrows():

    df = complete_df[complete_df['codice'] == row['Codice']]
        
    chart = create_vegalite_chart(df)

    color1, opac = get_color(df)

    marker = folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']], 
        popup=folium.Popup().add_child(folium.VegaLite(
        chart,
        width = '80%',
        height = '30%')),
        radius = 8,
        color = color1,
        weight = 0,
        fill = True,
        fill_opacity = abs(opac)+.2,
        tooltip=[row['Codice'], opac]
    )

    marker.add_to(m)


m.save('map.html')

In [13]:
flagged_table = pd.DataFrame()
flagged_filepath = "C:\\Users\\Andrew\\Desktop\\gw_study\\girotto_lab\\processed_data\\all_matrix\\flagged-tables"
for files in os.listdir(flagged_filepath):
    df = pd.read_excel(flagged_filepath + "\\" + files)
    df = df.iloc[[0], 1:].T
    flagged_table = flagged_table.append(df)
flagged_table = flagged_table.reset_index()

In [14]:
flagged_table = flagged_table.merge(df_subset, left_on='index', right_on='Codice', how = 'inner')

In [15]:
flagged_table

,index,0,Longitude,Latitude,Codice
0,PO0190030U0035,2.0,9.882081,45.244615,PO0190030U0035
1,PO0190210U0218,2.0,10.442024,44.974490,PO0190210U0218
2,PO019027NRA001,2.0,9.948778,45.291509,PO019027NRA001
3,PO019031NU0313,2.0,10.272853,45.085487,PO019031NU0313
4,PO019033NRA001,2.0,10.102293,45.207082,PO019033NRA001
...,...,...,...,...,...
345,919,2.0,11.648065,45.059868,919
346,920,2.0,11.648065,45.059868,920
347,921,2.0,11.700145,45.014134,921
348,922,2.0,11.700145,45.014134,922


In [16]:
def flag_color(x):
    if x == 2:
        return 'blue'
    if x == 1:
        return 'yellow'
    if x == 0:
        return 'red'
    else:
        return 'black'

In [17]:
m2 = folium.Map(location=[df_subset['Latitude'].mean(), df_subset['Longitude'].mean()], zoom_start=6)

for _, row in flagged_table.iterrows():

    df = complete_df[complete_df['codice'] == row['index']]
        
    chart = create_vegalite_chart(df)

    color1 = flag_color(row[0])

    marker = folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']], 
        popup=folium.Popup().add_child(folium.VegaLite(
        chart,
        width = '80%',
        height = '30%')),
        radius = 8,
        color = color1,
        weight = 0,
        fill = True,
        fill_opacity = .33,
        tooltip=[row['index'], 1]
    )

    marker.add_to(m2)


m2.save('map2.html')